In [23]:
# %pip install jupyter-dash -q

In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import numpy as np
from time import time
import plotly.graph_objects as go

### Rotation matrix from axis and angle

$$
R={\begin{bmatrix}\cos \theta +u_{x}^{2}\left(1-\cos \theta \right)&u_{x}u_{y}\left(1-\cos \theta \right)-u_{z}\sin \theta &u_{x}u_{z}\left(1-\cos \theta \right)+u_{y}\sin \theta \\u_{y}u_{x}\left(1-\cos \theta \right)+u_{z}\sin \theta &\cos \theta +u_{y}^{2}\left(1-\cos \theta \right)&u_{y}u_{z}\left(1-\cos \theta \right)-u_{x}\sin \theta \\u_{z}u_{x}\left(1-\cos \theta \right)-u_{y}\sin \theta &u_{z}u_{y}\left(1-\cos \theta \right)+u_{x}\sin \theta &\cos \theta +u_{z}^{2}\left(1-\cos \theta \right)\end{bmatrix}}
$$

### Rotation matrix from angle
$$
{\begin{aligned}R=R_{z}(\alpha )\,R_{y}(\beta )\,R_{x}(\gamma )&={\overset {\text{yaw}}{\begin{bmatrix}\cos \alpha &-\sin \alpha &0\\\sin \alpha &\cos \alpha &0\\0&0&1\\\end{bmatrix}}}{\overset {\text{pitch}}{\begin{bmatrix}\cos \beta &0&\sin \beta \\0&1&0\\-\sin \beta &0&\cos \beta \\\end{bmatrix}}}{\overset {\text{roll}}{\begin{bmatrix}1&0&0\\0&\cos \gamma &-\sin \gamma \\0&\sin \gamma &\cos \gamma \\\end{bmatrix}}}\\&={\begin{bmatrix}\cos \alpha \cos \beta &\cos \alpha \sin \beta \sin \gamma -\sin \alpha \cos \gamma &\cos \alpha \sin \beta \cos \gamma +\sin \alpha \sin \gamma \\\sin \alpha \cos \beta &\sin \alpha \sin \beta \sin \gamma +\cos \alpha \cos \gamma &\sin \alpha \sin \beta \cos \gamma -\cos \alpha \sin \gamma \\-\sin \beta &\cos \beta \sin \gamma &\cos \beta \cos \gamma \\\end{bmatrix}}\end{aligned}}
$$

In [9]:
def angles2Matrix(alpha, beta, gamma):
    """
    Convert Euler angles to rotation matrix.
    """
    alpha = np.radians(alpha)
    beta = np.radians(beta)
    gamma = np.radians(gamma)
    Rz = np.array([
        [np.cos(alpha), -np.sin(alpha), 0],
        [np.sin(alpha), np.cos(alpha), 0],
        [0, 0, 1]
    ])
    Ry = np.array([
        [np.cos(beta), 0, np.sin(beta)],
        [0, 1, 0],
        [-np.sin(beta), 0, np.cos(beta)]
    ])
    Rx = np.array([
        [1, 0, 0],
        [0, np.cos(gamma), -np.sin(gamma)],
        [0, np.sin(gamma), np.cos(gamma)]
    ])
    return Rz @ Ry @ Rx
#     return np.array([
#         [np.cos(alpha)*np.cos(beta), np.cos(alpha)*np.sin(beta)*np.sin(gamma)-np.sin(alpha)*np.cos(gamma), np.cos(alpha)*np.sin(beta)*np.cos(gamma)+np.sin(alpha)*np.sin(gamma)],
#         [np.sin(alpha)*np.cos(beta), np.sin(alpha)*np.sin(beta)*np.sin(gamma)+np.cos(alpha)*np.cos(gamma), np.sin(alpha)*np.sin(beta)*np.cos(gamma)-np.cos(alpha)*np.sin(gamma)],
#         [-np.sin(beta), np.cos(beta)*np.sin(gamma), np.cos(beta)*np.cos(gamma)]
#     ])

### Scaling matrix
$$
S_{v}={\begin{bmatrix}v_{x}&0&0\\0&v_{y}&0\\0&0&v_{z}\\\end{bmatrix}}
$$

In [ ]:
# Scaling matrix
def scaling_matrix(scale_x, scale_y, scale_z):
    return np.diag([scale_x, scale_y, scale_z])

### eigen decomposition of a covariance matrix

$$
\Sigma=RSS^TR^T
$$

In [10]:
# Covariance matrix
def covariance_matrix(rot, scale):
    return rot @ scale @ scale.T @ rot.T 

### Multivariate Gaussian Distribution

the probability density function (pdf) of a k dimension multivariate distribution is


$$
p(\mathbf{x}; \mu, \Sigma )={\frac {1}{\sqrt {(2\pi )^{k}|{\boldsymbol {\Sigma }}|}}}\exp \left(-{1 \over 2}(\mathbf {x} -{\boldsymbol {\mu }})^{\rm {T}}{\boldsymbol {\Sigma }}^{-1}({\mathbf {x} }-{\boldsymbol {\mu }})\right)
$$


In [40]:
def plot_ellipsoid(mean, cov, n_points=40):
    # SVD decomposition
    U, s, _ = np.linalg.svd(cov)
    eigvals = s
    eigvecs = U

    # Define a grid for the ellipsoid
    u = np.linspace(0, 2 * np.pi, n_points)
    v = np.linspace(0, np.pi, n_points)
    x = np.outer(np.cos(u), np.sin(v))
    y = np.outer(np.sin(u), np.sin(v))
    z = np.outer(np.ones_like(u), np.cos(v))

    # Combine the points into a single matrix for transformation
    ellipsoid_points = np.array([x.ravel(), y.ravel(), z.ravel()]).T
    transformed_points = ellipsoid_points @ np.diag(np.sqrt(eigvals)) @ eigvecs.T

    # Reshape the points back to the ellipsoid grid
    x = transformed_points[:, 0].reshape(n_points, n_points) + mean[0]
    y = transformed_points[:, 1].reshape(n_points, n_points) + mean[1]
    z = transformed_points[:, 2].reshape(n_points, n_points) + mean[2]

    return x, y, z

## Initialization

In [51]:
def get_gaussian_ellipsoid(mean_x, mean_y, mean_z, alpha, beta, gamma, scale_x, scale_y, scale_z):
    # Compute mean
    mean = np.array([mean_x, mean_y, mean_z])
    # Compute rotation matrix
    rot = rotation_matrix(alpha, beta, gamma)
    # Compute scaling matrix
    scale = scaling_matrix(scale_x, scale_y, scale_z)
    # Compute new covariance matrix
    cov = covariance_matrix(rot, scale)
    # Plot ellipsoid
    x, y, z = plot_ellipsoid(mean, cov, n_points)
    return x, y, z

n_points = 40
start = time()
x, y, z = get_gaussian_ellipsoid(0, 0, 0, 0, 0, 0, 1, 1, 1)
print(f"Computation time: {(time()-start)*1000:.2f}ms")


Computation time: 0.87ms


## Visualization

In [68]:
fig = go.Figure()
fig.add_trace(go.Surface(x=x, y=y, z=z))
fig.update_layout(
    title='Multivariate Gaussian Distribution',
    autosize=False,
    width=600,
    height=600,
    scene=dict(
        xaxis=dict(range=[-5, 5]),
        yaxis=dict(range=[-5, 5]),
        zaxis=dict(range=[-5, 5]),
        aspectratio=dict(x=1, y=1, z=1),
    ),
)

fig2D = go.Figure()
fig2D.add_trace(go.Contour(
        z=[[10, 10.625, 12.5, 15.625, 20],
           [5.625, 6.25, 8.125, 11.25, 15.625],
           [2.5, 3.125, 5., 8.125, 12.5],
           [0.625, 1.25, 3.125, 6.25, 10.625],
           [0, 0.625, 2.5, 5.625, 10]],
        # heatmap gradient coloring is applied between each contour level
        contours_coloring='heatmap' # can also be 'lines', or 'none'
    ))
fig2D.update_layout(
    title='Multivariate Gaussian Distribution',
    autosize=False,
    width=600,
    height=600,
    scene=dict(
        xaxis=dict(range=[-5, 5]),
        yaxis=dict(range=[-5, 5]),
        zaxis=dict(range=[-5, 5]),
        aspectratio=dict(x=1, y=1, z=1),
    ),
)


app = dash.Dash(__name__)
app.layout = html.Div([
    html.Div(children=[
        html.Div(children=[
            dcc.Graph(id='ellipsoid-plot', figure=fig),
        ], style={'display': 'inline-block'}),
        html.Div(children=[
            dcc.Graph(id='ellipsoid-plot2d', figure=fig2D),
        ], style={'display': 'inline-block'}),
    ], style={ 'display': 'inline-block'}),
    html.Div(children=[
        html.Div('mean-x', style={'color': 'white', 'fontSize': 14, "padding-left": "20px"}),
        dcc.Slider(id='mean-x', min=-4, max=4, step=0.1, value=0, marks={i: str(i) for i in range(-10, 11)}, updatemode='drag'),
        html.Div('mean-y', style={'color': 'white', 'fontSize': 14, "padding-left": "20px"}),
        dcc.Slider(id='mean-y', min=-4, max=4, step=0.1, value=0, marks={i: str(i) for i in range(-10, 11)}, updatemode='drag'),
        html.Div('mean-z', style={'color': 'white', 'fontSize': 14, "padding-left": "20px"}),
        dcc.Slider(id='mean-z', min=-4, max=4, step=0.1, value=0, marks={i: str(i) for i in range(-10, 11)}, updatemode='drag'),
        html.Div('alpha', style={'color': 'white', 'fontSize': 14, "padding-left": "20px"}),
        dcc.Slider(id='alpha', min=0, max=360, step=1, value=0, marks={i: str(i) for i in range(0, 361, 30)}, updatemode='drag'),
        html.Div('beta', style={'color': 'white', 'fontSize': 14, "padding-left": "20px"}),
        dcc.Slider(id='beta', min=0, max=360, step=1, value=0, marks={i: str(i) for i in range(0, 361, 30)}, updatemode='drag'),
        html.Div('gamma', style={'color': 'white', 'fontSize': 14, "padding-left": "20px"}),
        dcc.Slider(id='gamma', min=0, max=360, step=1, value=0, marks={i: str(i) for i in range(0, 361, 30)}, updatemode='drag'),
        html.Div('scale-x', style={'color': 'white', 'fontSize': 14, "padding-left": "20px"}),
        dcc.Slider(id='scale-x', min=0, max=5, step=0.1, value=1, marks={i: str(i) for i in range(0, 6)}, updatemode='drag'),
        html.Div('scale-y', style={'color': 'white', 'fontSize': 14, "padding-left": "20px"}),
        dcc.Slider(id='scale-y', min=0, max=5, step=0.1, value=1, marks={i: str(i) for i in range(0, 6)}, updatemode='drag'),
        html.Div('scale-z', style={'color': 'white', 'fontSize': 14, "padding-left": "20px"}),
        dcc.Slider(id='scale-z', min=0, max=5, step=0.1, value=1, marks={i: str(i) for i in range(0, 6)}, updatemode='drag'),
    ], style={'width': '30%', 'display': 'inline-block'}),
])

@app.callback(
    Output('ellipsoid-plot', 'extendData'),
    [Input('mean-x', 'value'),
     Input('mean-y', 'value'),
     Input('mean-z', 'value'),
     Input('alpha', 'value'),
     Input('beta', 'value'),
     Input('gamma', 'value'),
     Input('scale-x', 'value'),
     Input('scale-y', 'value'),
     Input('scale-z', 'value')]
)
def update_ellipsoid(*data):
    x, y, z = get_gaussian_ellipsoid(*data)
    return [{
        'x': [x],
        'y': [y],
        'z': [z]
    }, [0], n_points]

app.run_server(debug=True)
